# *Linear regression with regularizers*

In [32]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [33]:

#Let us explore the data and the description 

#Get the data from the web 
auto_data = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/auto-mpg/auto-mpg.data', delim_whitespace=True, index_col=False, header=None)

#print the first few rows of the data 
auto_data.head()

,0,1,2,3,4,5,6,7,8
0,18.0,8,307.0,130.0,3504.0,12.0,70,1,chevrolet chevelle malibu
1,15.0,8,350.0,165.0,3693.0,11.5,70,1,buick skylark 320
2,18.0,8,318.0,150.0,3436.0,11.0,70,1,plymouth satellite
3,16.0,8,304.0,150.0,3433.0,12.0,70,1,amc rebel sst
4,17.0,8,302.0,140.0,3449.0,10.5,70,1,ford torino


In [34]:
auto_data.shape

(398, 9)

In [35]:
auto_data.describe()

,0,1,2,4,5,6,7
count,398.000000,398.000000,398.000000,398.000000,398.000000,398.000000,398.000000
mean,23.514573,5.454774,193.425879,2970.424623,15.568090,76.010050,1.572864
std,7.815984,1.701004,104.269838,846.841774,2.757689,3.697627,0.802055
min,9.000000,3.000000,68.000000,1613.000000,8.000000,70.000000,1.000000
25%,17.500000,4.000000,104.250000,2223.750000,13.825000,73.000000,1.000000
50%,23.000000,4.000000,148.500000,2803.500000,15.500000,76.000000,1.000000
75%,29.000000,8.000000,262.000000,3608.000000,17.175000,79.000000,2.000000
max,46.600000,8.000000,455.000000,5140.000000,24.800000,82.000000,3.000000


In [36]:

auto_data.columns = ['mpg', 'cylinders', 'displacement','horsepower', 'weight','acceleration','model year','origin','car name']


#check by printing the data again
auto_data.head()
     

,mpg,cylinders,displacement,horsepower,weight,acceleration,model year,origin,car name
0,18.0,8,307.0,130.0,3504.0,12.0,70,1,chevrolet chevelle malibu
1,15.0,8,350.0,165.0,3693.0,11.5,70,1,buick skylark 320
2,18.0,8,318.0,150.0,3436.0,11.0,70,1,plymouth satellite
3,16.0,8,304.0,150.0,3433.0,12.0,70,1,amc rebel sst
4,17.0,8,302.0,140.0,3449.0,10.5,70,1,ford torino


In [37]:

# ##Counting number of missing values in each column
# for col in auto_mpg_data.columns.values:
#   val = set(auto_mpg_data[col])
#   null_val = sum(pd.isna(auto_mpg_data[col]))
#   print(col + ': ' + str(auto_mpg_data[col].dtypes) + ', ' + str(len(val)) + ' unique values, ' + str(null_val) + ' null values!')

# # auto_mpg_data.isnull().sum()
print(auto_data.isnull().sum())
auto_data.nunique()



mpg             0
cylinders       0
displacement    0
horsepower      0
weight          0
acceleration    0
model year      0
origin          0
car name        0
dtype: int64


mpg             129
cylinders         5
displacement     82
horsepower       94
weight          351
acceleration     95
model year       13
origin            3
car name        305
dtype: int64

In [38]:
auto_data = auto_data.replace('?' , np.nan) # replacing '?' values by nan values

In [39]:
print(auto_data.isnull().sum())

mpg             0
cylinders       0
displacement    0
horsepower      6
weight          0
acceleration    0
model year      0
origin          0
car name        0
dtype: int64


In [40]:
auto_data = auto_data.dropna()
print(auto_data.isnull().sum())

print(auto_data.shape)


mpg             0
cylinders       0
displacement    0
horsepower      0
weight          0
acceleration    0
model year      0
origin          0
car name        0
dtype: int64
(392, 9)


In [41]:
auto_data.drop('car name', axis=1, inplace=True)

In [42]:
auto_data.head()

,mpg,cylinders,displacement,horsepower,weight,acceleration,model year,origin
0,18.0,8,307.0,130.0,3504.0,12.0,70,1
1,15.0,8,350.0,165.0,3693.0,11.5,70,1
2,18.0,8,318.0,150.0,3436.0,11.0,70,1
3,16.0,8,304.0,150.0,3433.0,12.0,70,1
4,17.0,8,302.0,140.0,3449.0,10.5,70,1


In [43]:
print(auto_data.dtypes)

mpg             float64
cylinders         int64
displacement    float64
horsepower       object
weight          float64
acceleration    float64
model year        int64
origin            int64
dtype: object


In [44]:
# converting all data type to float

auto_data = auto_data.astype(float)
print(auto_data.dtypes)

mpg             float64
cylinders       float64
displacement    float64
horsepower      float64
weight          float64
acceleration    float64
model year      float64
origin          float64
dtype: object


## making a train Test Validation split

In [45]:
auto_data_train_initial = auto_data.sample(frac=0.8, random_state=200)
auto_data_test = auto_data.drop(auto_data_train_initial.index)

print(len(auto_data_train_initial))
print(len(auto_data_test))

314
78


In [46]:
auto_train = auto_data_train_initial.sample(frac=0.8, random_state=200)

auto_val = auto_data_train_initial.drop(auto_train.index)

len(auto_train) , len(auto_data_test), len(auto_val)

(251, 78, 63)

## *Computing beta for this auto_data*

In [47]:
# saving the length data

n_train = len(auto_train)
n_val = len(auto_val)
n_test = len(auto_data_test)

print(n_train,n_val,n_test)

251 63 78


In [48]:
cols = [1,2,3,4,5,6,7]

x_train = auto_train[auto_train.columns[cols]]
x_val = auto_val[auto_val.columns[cols]]
x_test = auto_data_test[auto_data_test.columns[cols]]

In [49]:
x_train

,cylinders,displacement,horsepower,weight,acceleration,model year,origin
28,8.0,304.0,193.0,4732.0,18.5,70.0,1.0
154,6.0,250.0,72.0,3432.0,21.0,75.0,1.0
337,4.0,107.0,72.0,2290.0,17.0,80.0,3.0
93,8.0,318.0,150.0,4237.0,14.5,73.0,1.0
109,4.0,140.0,72.0,2401.0,19.5,73.0,1.0
...,...,...,...,...,...,...,...
371,4.0,135.0,84.0,2525.0,16.0,82.0,1.0
12,8.0,400.0,150.0,3761.0,9.5,70.0,1.0
379,4.0,98.0,70.0,2125.0,17.3,82.0,1.0
199,6.0,225.0,100.0,3651.0,17.7,76.0,1.0


In [50]:
# converting to numpy array

x_train = x_train.to_numpy()
x_val = x_val.to_numpy()
x_test = x_test.to_numpy()

In [51]:
x_train

array([[  8. , 304. , 193. , ...,  18.5,  70. ,   1. ],
       [  6. , 250. ,  72. , ...,  21. ,  75. ,   1. ],
       [  4. , 107. ,  72. , ...,  17. ,  80. ,   3. ],
       ...,
       [  4. ,  98. ,  70. , ...,  17.3,  82. ,   1. ],
       [  6. , 225. , 100. , ...,  17.7,  76. ,   1. ],
       [  6. , 146. , 120. , ...,  13.8,  81. ,   3. ]])

In [52]:
# adding row of one at the end 

x_train = np.hstack((x_train , np.ones((x_train.shape[0], 1), dtype=x_train.dtype)))
x_test = np.hstack((x_test, np.ones((x_test.shape[0], 1), dtype=x_test.dtype)))
x_val = np.hstack((x_val , np.ones((x_val.shape[0], 1), dtype=x_val.dtype)))

response column

In [53]:


y_train = auto_train[auto_train.columns[0]].to_numpy()
y_test = auto_data_test[auto_data_test.columns[0]].to_numpy()
y_val = auto_val[auto_val.columns[0]].to_numpy()

finding the beta

In [54]:
XTX = np.matmul(np.transpose(x_train),x_train)

print('XTX shape ', XTX.shape)


XTX shape  (8, 8)


In [59]:
y_train.shape

(251,)

In [61]:
y_train = y_train.reshape(251,1)

In [62]:
XTY = np.matmul(np.transpose(x_train),y_train)
print('XTY shape', XTY.shape)


XTY shape (8, 1)


In [63]:
beta = np.linalg.solve(XTX,XTY)

#beta = np.matmul(np.linalg.inv(XTX),XTY)

print('beta', beta)

beta [[-9.25331740e-01]
 [ 1.58272306e-02]
 [-2.18273959e-02]
 [-5.49403048e-03]
 [-1.88172751e-01]
 [ 7.43821476e-01]
 [ 9.98127605e-01]
 [-1.13539831e+01]]


In [64]:
print(np.linalg.matrix_rank(XTX))

8


checking weather the XTX * XTX_inv is identity matrix ornot

In [68]:
XTX_inv = np.linalg.inv(XTX)

p = (np.matmul(XTX , XTX_inv))

is_identiy = np.all(p == np.identity(XTX.shape[0]))
print(is_identiy)

False


so no it is not identity matrix

$\textbf{Note:}$ Because the matrix $X^\top X$ is full-rank, that is, $8:= \text{rank}(X^\top X)=8$, we see that $\texttt{numpy}$ does not raise any error while computing the inverse of $X^\top X$. But, if it would have been not a full rank matrix, numpy would have raised an error. Hence we will use $\texttt{scipy}$ to solve for $\beta$ and check if we get any error or not. 

In [69]:
import scipy.linalg

beta_scipy = scipy.linalg.solve(XTX,XTY)

In [70]:
beta_scipy

array([[-9.25331740e-01],
       [ 1.58272306e-02],
       [-2.18273959e-02],
       [-5.49403048e-03],
       [-1.88172751e-01],
       [ 7.43821476e-01],
       [ 9.98127605e-01],
       [-1.13539831e+01]])

In [71]:
print(np.linalg.cond(XTX))

7311294161.881962


We can see that the ill-conditioning of $X^\top X$ might lead to wild changes in solutions of linear regression parameters, even for small changes in the output $\mathbf{y}$. 

# Adding $\ell_2$ regularizer to improve the conditioning of the matrix: 

We shall motivate the use of $\ell_2$ regularizer to improve the conditioning of the matrix. 

Instead of minimizing the original OLS objective 
$
\begin{align}
L_{OLS}(\beta_0, \beta_1,\ldots,\beta_d) = \sum_{i=1}^{n} [y^i - ( \beta_0 + \sum_{j=1}^{d} \beta_j x_j^i) ]^2. 
\end{align}
$ 

we shall now minimize the OLS objective added with a scaled $\ell_2$ regularizer. 

The $\ell_2$ regularizer in $\beta \in {\mathbb{R}}^{d+1}$ is defined as the squared $\ell_2$ norm of $\beta$: 

$
\begin{align}
\|\beta\|_2^2 = \sum_{i=1}^{d+1} \beta_i^2.  
\end{align}
$

Hence we shall now solve: 
$
\begin{align}
L_{\text{ridge}}(\beta_0, \beta_1,\ldots,\beta_d) = \sum_{i=1}^{n} [y^i - ( \beta_0 + \sum_{j=1}^{d} \beta_j x_j^i) ]^2 + \lambda \sum_{i=1}^{d+1} \beta_i^2. 
\end{align}
$ 

The above objective function is called $\textbf{ridge}$ regression objective. $\lambda>0$ is a regularization hyperparameter. 



Now we can write the objective function as:

$
L_{\text{ridge}}(\beta) = \|\mathbf{y} - \mathbf{X}\mathbf{\beta} \|_F^2 + \lambda \|\beta\|_2^2.
$

To solve 

$
\min_\beta L_{\text{ridge}}(\beta) = \|\mathbf{y} - \mathbf{X}\mathbf{\beta} \|_F^2 + \lambda \|\beta\|_2^2, 
$
we find the gradient with respect to $\beta$ and equate to zero. 

Thus we get:

$
\begin{align}
\nabla_\beta L_{\text{ridge}}(\beta) &= \mathbf{0} \\ 
\implies -\mathbf{X}^\top \mathbf{y} + \mathbf{X}^\top\mathbf{X} \beta + \lambda I\beta &= \mathbf{0} \\ 
\implies \beta &= (\mathbf{X}^\top\mathbf{X}+\lambda I)^{-1} \mathbf{X}^\top \mathbf{y}. 
\end{align}
$

Note that the closed form expression for $\beta$ is always valid since $(\mathbf{X}^\top\mathbf{X}+\lambda I)$ is invertible.  
